In [205]:
import twitter as tw
from TwitterSearch import *
import pandas as pd
import os, sys, time, operator, re, plotly, pyspark
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [217]:
# spark args
SUBMIT_ARGS = "--packages com.datastax.spark:spark-cassandra-connector_2.11:2.0.1  pyspark-shell" 
SPARK_HOME = "/home/dsr/servers/spark"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["PYSPARK_PYTHON"] = "/home/dsr/servers/jupyter/bin/anaconda3/bin/python"

#zamkniecie starego sparkcontextu
sc = pyspark.SparkContext.getOrCreate()
sc.stop()

#start spark
conf = pyspark.SparkConf()
conf.setAppName('MySpark')
conf.setMaster("spark://3-ds-lookalike-st:7077")
# conf.setMaster("spark://1-oset:7077")

conf.set("spark.cassandra.connection.port", '10042')
conf.set("spark.submit.deployMode",'cluster')
conf.setSparkHome(SPARK_HOME)

# conf.set("ClusterOne/spark.cassandra.connection.host", '4-oset')
conf.set("ClusterTwo/spark.cassandra.connection.host", '3-ds-lookalike-st')

sc = pyspark.SparkContext(conf=conf)
sqlContext = pyspark.sql.SQLContext(sc)

## metoda wyszukiwania danego słowa w tweetach

In [219]:
columns = ['username','tweet_text', 'time', 'language']
tabelka = pd.DataFrame(columns=columns)
tabelka

,username,tweet_text,time,language


In [221]:
try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['google', 'youtube'], or_operator=True)#,['google']) # let's define all words we would like to have a look for
    tso.add_keyword(['google', 'youtube'], or_operator=True)
    #tso.set_language('pl') # we want to see German tweets only
    tso.set_include_entities(False) # and don't give us all those entity information
    # it's about time to create a TwitterSearch object with our secret tokens
    ts = TwitterSearch(
        consumer_key = 'WMIZHIKwq3pxCeNlRZWIxEmLj',
        consumer_secret = 'ETfyTs28mGYypj9X0d7PeVBtLVUCA04G0QH3SM9lVcZ8d6doxr',
        access_token = '981943339686744065-174h6G2ZVXPPyNA4Yv4zIyMFaUlEXkR',
        access_token_secret = 'yGROxnMt2ACBKpaglApUSRUlvsbGwKEpViSVcFoW81G7u'
     )

     # this is where the fun actually starts :)
    for tweet in ts.search_tweets_iterable(tso):
        t3 = time.time()
        print(( tweet['user']['screen_name'], tweet['text'], tweet['created_at'] , tweet['lang']))
        a = (tweet['user']['screen_name'], tweet['text'], tweet['created_at'] , tweet['lang'])
        token = list(a)
        A = {'username':token[0], 'tweet_text':token[1], 'time':token[2], 'language':token[3]}
        tabelka = tabelka.append(A, ignore_index=True)
        le = len(tabelka)
        t4 = time.time()
        if(le==500):
            print("teraz"+str(t4-t3))
            t1 = time.time()
            PD_TAB = sqlContext.createDataFrame(tabelka)
            PD_TAB.write.format("org.apache.spark.sql.cassandra").mode('append').options(cluster = "ClusterTwo",table="masterset", keyspace="look_alike_learning").save()
            columns = ['username','tweet_text', 'time', 'language']
            tabelka = pd.DataFrame(columns=columns)
            t2 = time.time()
            print(t2-t1)
    PD_TAB = sqlContext.createDataFrame(tabelka)
    PD_TAB.write.format("org.apache.spark.sql.cassandra").mode('append').options(cluster = "ClusterTwo",table="masterset", keyspace="look_alike_learning").save()
    columns = ['username','tweet_text', 'time', 'language']
    tabelka = pd.DataFrame(columns=columns)
    time.sleep(5)
except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

('japan0090', 'RT @No_namapo: 在日特権のタブーに踏み込む安倍総理！通名制度はおかしい！外国人献金で晒される通名の存在\nhttps://t.co/cydHc4Cqmb\n安倍さん！日本をお願いします。\n#在日特権\u3000#在日特権廃止\u3000#通名廃止', 'Sat Apr 07 11:14:18 +0000 2018', 'ja')
('bngtxnps', 'RT @bts_bighit: [#방탄밤] 아미와 함께 해서 더 즐거웠던 #고민보다Go 사전 녹화! #마이크를뚫고나오는아미의응원소리 #탕진잼탕진잼탕진잼 #긋냥긋냥\n(https://t.co/idEsfo8MLy)', 'Sat Apr 07 11:14:18 +0000 2018', 'ko')
('okoge_star', '弱酸性ミリオンアーサー #45 https://t.co/jl5ebp4zPB\n\nやっぱ富豪っち好きだわ(', 'Sat Apr 07 11:14:18 +0000 2018', 'ja')
('sankoichi351hp', 'RT @sankoiti_offici: 【大切なお話】さんこいちが新しく生まれ変わります。\n\n🌟続きはYouTube↓↓↓🌟\nhttps://t.co/YZUCaGUP8T\n\nチャンネル登録もよろしく🙏 https://t.co/1mA0Lxiwdn', 'Sat Apr 07 11:14:18 +0000 2018', 'ja')
('NorbertRoseau', 'J\'aime une vidéo @YouTube\xa0: "The Beatles / Norwegian Wood【Original】" à l\'adresse https://t.co/RbaKVYyXkM.', 'Sat Apr 07 11:14:18 +0000 2018', 'fr')
('ymgn_ll', 'RT @hololivetv: ホロライブをスマートフォンでVTuberになりきれるアプリとしてアップデートしました。Android版も同時リリースです。まだ、機能は少ないですが、ぜひお試しください！\n\nhttps://t.co/yI9eIGdCh9\nhttps://t.co/…',

('SamSekund', 'RT @dokolenko_mk: Ten Sharp - You https://t.co/Pvd9dy56PH via @YouTube', 'Sat Apr 07 11:14:16 +0000 2018', 'en')
('A_ae_16', 'RT @d3wadelm: 💡مسابقة #حملة_أرحنا_بالصلاة 🕌\n\n7⃣ • اليوم السابع • 7⃣\n🔰( المسار التقني )🔰\n\nأجب على السؤال بعد مشاهدة  المقطع التالي 🎞\n\nhttps:…', 'Sat Apr 07 11:14:16 +0000 2018', 'ar')
('amassogera', 'Ａマッソが人の言うこと聞くように矯正する回\n\nhttps://t.co/gBCna2Y282\n\n#Ａマッソ #ゲラニチョビ https://t.co/6E4z1JvjxR', 'Sat Apr 07 11:14:16 +0000 2018', 'ja')
('sh7if', 'nice and very helpful twitter automation tool!\n#RoboTwity\nhttps://t.co/PhMq3zxd5t', 'Sat Apr 07 11:14:16 +0000 2018', 'en')
('akane_3104_', 'RT @miztamari_nikki: サブチャンネル\n\n【24時間モニタリング】水溜りハウスのキッチンに1日中監視カメラ置いてみた！\nhttps://t.co/gLn7o6gEpw\n遅くなってごめんね。僕ら出てない時間も多いので、飛ばしながらとかでもいいので時間ある時に見て…', 'Sat Apr 07 11:14:16 +0000 2018', 'ja')
('amydia_tony', 'I added a video to a @YouTube playlist https://t.co/CfV0ZdiQgY 보정 No! 모공, 패인 여드름흉터, 요철 완벽커버', 'Sat Apr 07 11:14:16 +0000 2018', 'ko')
('_hachi_ni_82_', 'RT @resold

('Desa_ExolIgot7', "RT @Groovechaja: 토요일\n\nEXO-CBX (첸백시) Blooming Days 'Playdate' #XIUMIN https://t.co/ET5e2KOLuD - @YouTube", 'Sat Apr 07 11:14:14 +0000 2018', 'ko')
('daisukihikaru22', 'RT @hikaru2nd1031: 拡散希望、リツイートお願いします。\n\nヒカルの名前を無断で使用しお金儲けをしている人がいる件について\nhttps://t.co/DnuOIXZDSA', 'Sat Apr 07 11:14:14 +0000 2018', 'ja')
('IpswichConnect', '#DottyDirectory #YouTube https://t.co/S1LDiUq6ia https://t.co/0u7ezf6zWa', 'Sat Apr 07 11:14:14 +0000 2018', 'und')
('pechi_pechi65', 'RT @antsa_marin23: こちらニコニコ↓\nhttps://t.co/b3LyzQJ8dF\n\nこちらYouTube↓\nhttps://t.co/NfgRKfntUK\n\nたくさん見てなー！！！ https://t.co/sRL9DBtPOy', 'Sat Apr 07 11:14:14 +0000 2018', 'ja')
('FuriaTotal898', 'Gostei de um vídeo do @YouTube https://t.co/4Dt093CvcR Diego Miranda Tomorrowland Belgium 2017', 'Sat Apr 07 11:14:14 +0000 2018', 'pt')
('LeeEuiJinhanda', 'RT @147_Company: [직캠]180407 #유앤비 #UNB 전체 신촌 게릴라 by 철이\n#더유닛 #THEUNIT #온리원 #OnlyOne\n#준 #의진 #고호정 #필독 #마르코 #지한솔 #대원 #기중 #찬\n@official_UN_B\nhttps…', 'Sat Apr 07 11:14:14

('AngyHat', 'RT @LDNEurovision: Tu Canción by @Alfred_ot2017 and @Amaia_ot2017 LIVE at the 2018 London Eurovision Party #Spain 🇪🇸#Almaia #AlmaiaLondres…', 'Sat Apr 07 11:14:12 +0000 2018', 'es')
('Nana_otaku4life', 'RT @btschartdata: #Euphoria has surpassed 10 MILLION views on Youtube. @BTS_twt', 'Sat Apr 07 11:14:12 +0000 2018', 'en')
('vladimir_rohal', 'VIDEOROHAĽ, AVANTI Dvi chmary - DISKO - ľudová pieseň: https://t.co/3Z1nPC8hWk na stránkach @YouTube', 'Sat Apr 07 11:14:12 +0000 2018', 'und')
('Denasadorable', 'GAME OVER | Enjoy The Show #AntiSchool #QAnon April 6 2018 https://t.co/ppwAbaTtI6 via @YouTube', 'Sat Apr 07 11:14:12 +0000 2018', 'en')
('thinkingmanpro', 'I liked a @YouTube video https://t.co/N5OD1XdQay Joseph Valtellini vs. Raymond Daniels (GLORY 13) | Fight Watch', 'Sat Apr 07 11:14:12 +0000 2018', 'en')
('yuridianaromer4', 'He añadido un vídeo a una lista de reproducción de @YouTube (https://t.co/2H7BUiz0Ke - E-girls / Y.M.C.A. (E-girls version) Music', 'Sat Apr 07 11

('Rishadhussain4', '@BibssShirley @YouTube @Saavn @ShirleySetia @Dj_Chetas @itsaadee @OnStageTalents Oooo congratulations @ShirleySetia… https://t.co/9qigFFNbAL', 'Sat Apr 07 11:14:10 +0000 2018', 'en')
('PonyzyxWu', 'RT @tratchanu: https://t.co/ZKpc6ube8o มาร์คบอก จาง PD เป็นคนที่ไม่หยิ่งเลย เป็นกันเองมาก เป็นคนที่เข้าใจความรู้สึกคนอื่นมาก แล้วก็ขยันมาก…', 'Sat Apr 07 11:14:10 +0000 2018', 'th')
('sadiizzz', 'RT @bts_bighit: [#방탄밤] 아미와 함께 해서 더 즐거웠던 #고민보다Go 사전 녹화! #마이크를뚫고나오는아미의응원소리 #탕진잼탕진잼탕진잼 #긋냥긋냥\n(https://t.co/idEsfo8MLy)', 'Sat Apr 07 11:14:10 +0000 2018', 'ko')
('esoshenxa', 'RT @NickiMVersess: Nicki Need to Sample this on song or feature or some with this beat or a harder beat  . I need her to Bodie them 🔥🔥🔥🔥 I…', 'Sat Apr 07 11:14:10 +0000 2018', 'en')
('mester_de_be', 'RT @karnogain: https://t.co/JbiGs5p83p', 'Sat Apr 07 11:14:10 +0000 2018', 'und')
('AKIRA_amimoti', 'RT @satomimi__: 【神回】味方か？敵か？敵に向かって突っ込んでいくテロ爺がヤバすぎたｗ【Dead by Daylight】\n\n↓動画はこちらから！↓\n https://t.co/mGlFbRY8GC\n

...

('lojadicord', 'Adicionei um vídeo a uma playlist @YouTube https://t.co/ZOGdlnZNkb axial y esqueleto apendicular - Medicina', 'Sat Apr 07 11:13:45 +0000 2018', 'pt')
('17_caratjw', "RT @boo_bit0116: boo-bit 2ND PHOTOBOOK\n\n' HEART FLUTTER '\n\n➖ \n\n💓전체 입금기간 : 2018.4.7 - 5.12\n\n💗입금 폼 : https://t.co/OR8bhp9gay\n\n#승관 #세븐틴 #seung…", 'Sat Apr 07 11:13:45 +0000 2018', 'ko')
('Black_secret05', 'RT @dearroby: ย่องมาปล่อยคลิปตอนดึกค่ะ ใครยังไม่นอนดูแล้วก็ขอให้ฝันดี ใครตื่นมาดูตอนเช้าก็ขอให้สดใสค่ะ 😆\nคลิปนี้เป็นคลิปเฌอแตม(สั้นๆ)+ความแ…', 'Sat Apr 07 11:13:45 +0000 2018', 'th')
('KingNamjoon912', 'RT @bts_bighit: [#방탄밤] 아미와 함께 해서 더 즐거웠던 #고민보다Go 사전 녹화! #마이크를뚫고나오는아미의응원소리 #탕진잼탕진잼탕진잼 #긋냥긋냥\n(https://t.co/idEsfo8MLy)', 'Sat Apr 07 11:13:45 +0000 2018', 'ko')
('hina_kagawa', 'RT @exrikariko: YouTube更新しました🔻\n【初心者2人が荒野行動に挑戦！】\nhttps://t.co/cvtUHVU4xx', 'Sat Apr 07 11:13:45 +0000 2018', 'ja')
('gloomysmeraldo', 'RT @btsanalytics: [MTV Miaw @BTS_twt: Vote Rate]\n\nTweets &amp; RTs in the last hour (6:

('155Sadia', 'RT @btsanalytics: [MTV Miaw @BTS_twt: Vote Rate]\n\nTweets &amp; RTs in the last hour (6:00pm - 7:00pm) → #PremiosMTVMiaw + #MTVBRKPOPBTS\n\n1. BTS…', 'Sat Apr 07 11:13:43 +0000 2018', 'en')
('keenanls', 'RT @ElizabethRBess: @seanhannity @Disney Pig, pervert &amp; racist....are you looking in the mirror Sean? And let’s add liar, brown noser &amp; tre…', 'Sat Apr 07 11:13:43 +0000 2018', 'en')
('alikaA_neehraF', 'Thavaratha needhi tharum engal boomi.. Irul padhai thedi ini pogum koodi...\nSuraiyadum sulchi kaadai thamizhar desa… https://t.co/giF8gLLQDw', 'Sat Apr 07 11:13:43 +0000 2018', 'hi')
('MihaltanAndrei', 'Am apreciat un videoclip pe @YouTube, https://t.co/Hz9fzPgDJv RANK 5 TOW TRUCK! CE FACEM PE VIITOR? | Admin Life (#423)', 'Sat Apr 07 11:13:43 +0000 2018', 'ro')
('exo_ismylife21', "RT @Groovechaja: 토요일\n\nEXO-CBX (첸백시) Blooming Days 'Playdate' #XIUMIN https://t.co/ET5e2KOLuD - @YouTube", 'Sat Apr 07 11:13:43 +0000 2018', 'ko')
('xtratonworld', 'JANE THE VIRGIN - 

('tatsu121218', 'RT @Dominique298: 【LIVE録画】FORTNITE チャレンジやってくー #10: https://t.co/0MbSQTLxnf、@YouTube がアップロード', 'Sat Apr 07 11:13:41 +0000 2018', 'ja')
('X_FRPM6_X', 'Youtubeにある｢勘違いしたヤツらwww｣みたいな動画の中の1枚。\n\nこの人若干つぐつぐに似てる気がする。この人がこの髪型やると叩かれるけど、松岡禎丞って言う名前だったら叩かれないんだろうなぁ… https://t.co/6McUgYpXTI', 'Sat Apr 07 11:13:41 +0000 2018', 'ja')
('giant1020', '전희경의 브리핑룸 (폭주하는 文정권, 선거에서 심판하자!) https://t.co/grXxaJWX3p', 'Sat Apr 07 11:13:41 +0000 2018', 'ko')
('P1raterus', 'Bloodborne™*Кормилица Мерго.: https://t.co/9eDIpfkXGS на @YouTube', 'Sat Apr 07 11:13:41 +0000 2018', 'bg')
('Z2FFF', "RT @hoverdoss: Le frérot nosta vient de nous balancer son dernier morceau agrémenté d'un très gros clip allez voir ça🚨🚨https://t.co/MD56Zmc…", 'Sat Apr 07 11:13:41 +0000 2018', 'fr')
('TheFrozys', 'J\'aime une vidéo @YouTube\xa0: "Fortnite - I win even while being Swatted | DrLupo" à l\'adresse https://t.co/OrbyCDr9LD.', 'Sat Apr 07 11:13:41 +0000 2018', 'fr')
('36garhia', 'जानिए आज का इतिहास https://t.co/TpfAstpce

('5FWb5MW0Tq3AbW9', 'RT @skypeace_teojin: コンビニに売ってた10種類のアイス全部入れて混ぜた味は？: https://t.co/P9XQ4KORXf、@YouTube', 'Sat Apr 07 11:13:39 +0000 2018', 'ja')
('saiikhim_m', 'RT @GOT7PROJECT: [💚🐥] มารวมพลังอากาเซช่วยกันโดเนทคนละเล็กละน้อย เพื่อโปรเจคในคอน #GOT7WorldTour2018InBangkok #GOT7WorldTour2018InBKK กันนะค…', 'Sat Apr 07 11:13:39 +0000 2018', 'th')
('superkindlove', 'RT @TeenVogue: The YouTube shooter’s horrific actions had nothing to do with her nationality or her religion. https://t.co/tcdLA7C06H', 'Sat Apr 07 11:13:39 +0000 2018', 'en')
('phurist', 'RT @NateNM9: #เฌอสิค #BNK48\nBNK48 x FUJIFILM X-A5 เบื้องหลังโฆษณาที่คุณไม่เคยเห็นมาก่อน https://t.co/yQ7Q6ac3qJ ข้อมูลจาก @YouTube', 'Sat Apr 07 11:13:39 +0000 2018', 'th')
('TRILLTALK', 'https://t.co/QBvJX3zq9H', 'Sat Apr 07 11:13:39 +0000 2018', 'und')
('melikeyexo', 'RT @smolkjd: wait what if we get another soft anniversary vlive and youtube video from EXO today like last year https://t.co/8kp5KV7cXM', 'Sat Apr 07 11:13:38 +0000 2018', 'e

KeyboardInterrupt: 

In [188]:
PD_TAB = sqlContext.createDataFrame(tabelka)
PD_TAB.write.format("org.apache.spark.sql.cassandra").mode('append').options(cluster = "ClusterTwo",table="masterset", keyspace="look_alike_learning").save()
PD_TAB = sqlContext.read.format("org.apache.spark.sql.cassandra").options(cluster = "ClusterTwo",table="masterset", keyspace="look_alike_learning").load()
PD_TAB.show()

KeyboardInterrupt: 

## dodanie czasu


In [172]:
tabelka = PD_TAB.toPandas()

In [173]:
t2 = tabelka.copy()

In [174]:
profilki = t2.groupby(['time']).size().reset_index(name='count')
profilki

,time,count
0,Fri Apr 06 00:00:05 +0000 2018,1
1,Fri Apr 06 00:03:12 +0000 2018,1
2,Fri Apr 06 00:04:04 +0000 2018,1
3,Fri Apr 06 00:04:08 +0000 2018,1
4,Fri Apr 06 00:05:24 +0000 2018,1
5,Fri Apr 06 00:05:31 +0000 2018,1
6,Fri Apr 06 00:06:47 +0000 2018,1
7,Fri Apr 06 00:07:35 +0000 2018,1
8,Fri Apr 06 00:10:38 +0000 2018,1
9,Fri Apr 06 00:12:57 +0000 2018,1


In [175]:
keyword='facebook'
trace0 = go.Bar(
    x=profilki['time'],
    y=profilki['count'],
    name='Primary Product',
    marker=dict(
        color='rgb(49,130,189)'
    )
)


data = [trace0]
layout = go.Layout(
    xaxis=dict(tickangle=-45),
    title='Liczba tweetów z keywordem '+str(keyword),
    barmode='group',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='angled-text-bar')